In [2]:
//选择的训练样本, 感谢小亮的《Google Earth Engine（监督分类）》
var LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')
print(LABEL_DATA);
function maskL8sr(image) {
  var cloudShadowBitMask = (1 << 3);
  var cloudsBitMask = (1 << 5);
  var qa = image.select('pixel_qa');
  var mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
                .and(qa.bitwiseAnd(cloudsBitMask).eq(0));
  return image.updateMask(mask);
}
var landsatImage = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
    .filterDate('2019-01-01', '2019-12-31')
    .map(maskL8sr)
    .mean();
// 使用下列波段作为特征
var bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'];
// 通过要素集在Landsat-8中选取样本，把landcover属性赋予样本
var training = landsatImage.select(bands).sampleRegions({
  collection: LABEL_DATA,
  properties: ['landcover'],
  scale: 30
});
// 做精度评价
// 在training要素集中增加一个random属性，值为0到1的随机数
var withRandom = training.randomColumn('random');
//把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
var split = 0.7; 
var trainingPartition = withRandom.filter(ee.Filter.lt('random', split));
var testingPartition = withRandom.filter(ee.Filter.gte('random', split));
// 利用70%数据随机森林进行训练
var trainedClassifier = ee.Classifier.randomForest(5).train({
  features: trainingPartition,
  classProperty: 'landcover',
  inputProperties: bands
});
//对Landsat-8进行分类
var class_img=landsatImage.select(bands).classify(trainedClassifier)
// 对测试数据进行预测
var test = testingPartition.classify(trainedClassifier);
// 制作混淆矩阵，并打印出来
var confusionMatrix = test.errorMatrix('landcover', 'classification');
print('Confusion Matrix', confusionMatrix);
Map.centerObject(LABEL_DATA);
Map.addLayer(class_img, {min: 0, max: 2, palette: ['red', 'green', 'blue']});

ReferenceError: ee is not defined